This notebook produces plots to illustrate metrics such as MCC, precision and recall. Curently the plot it will produce are confusion matrices and precision-recall curves. The plots are made for one combination of experiment group / threshold strategy / data split

In [ ]:
import os
import pandas as pd
from mtqe.data.loaders import load_ced_test_data
from mtqe.utils.paths import EVAL_DIR, PREDICTIONS_DIR
from mtqe.utils.plots import create_confusion_matrix_plot, create_precision_recall_curve
from mtqe.utils.language_pairs import LI_LANGUAGE_PAIRS_WMT_21_CED

In [ ]:
# Set the parameters to be used in the rest of the notebook

EXPERIMENT_GROUP_NAME = 'wmt21_annotator'
LLM = True
DATA_SPLIT = 'test'
THRESHOLD_STRATEGY = 'default'
FILE_SUFFIX = 'median_results.csv'
CONFUSION_MATRIX_NAME = 'Trained Model - Multilingual Authentic Data'
PR_PLOT_NAME = 'Trained Model - Multilingual Authentic Data'

In [ ]:
# Identify folders of evaluation data for the experiment group - the enja evaluations might be in a separate folder.
eval_files = [os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME, file) for file in os.listdir(os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME)) if file.endswith(FILE_SUFFIX)]
enja_separate = False
if os.path.isdir(os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME + '_enja')):
    eval_files.extend([os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME + '_enja', file) for file in os.listdir(os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME + '_enja')) if file.endswith(FILE_SUFFIX)])
    enja_separate = True
    

In [ ]:
# Load the evaluation results into a dataframe
dfs = []
for file in eval_files:
    df = pd.read_csv(file)
    dfs.append(df)

df_eval_results = pd.concat(dfs)

In [ ]:
# Filter the results on threshold strategy and data split
df_eval_results = df_eval_results[(df_eval_results['threshold_strategy'] == THRESHOLD_STRATEGY) & (df_eval_results['split'] == DATA_SPLIT)]

In [ ]:
# Get lists of data to create plot
# NOTE: The file_prefix needs to be changed if an LLM prompt is to be evaluated - this could be make more
# robust in the code so that a manual change is not necessary.
preds = []
preds_probs = []
targets = []
cm_titles = [] # titles for confusion matrix plots
pr_titles = [] # titles for precision-recall plots
for lp in LI_LANGUAGE_PAIRS_WMT_21_CED:
    seed = df_eval_results[df_eval_results['language_pair']==lp].iloc[0]['seed']
    mcc = df_eval_results[df_eval_results['language_pair']==lp].iloc[0]['MCC']
    precision = df_eval_results[df_eval_results['language_pair']==lp].iloc[0]['precision']
    recall = df_eval_results[df_eval_results['language_pair']==lp].iloc[0]['recall']
    cm_titles.append('Language pair:' + lp.upper() + '\nMCC:' + str(mcc.round(3)) + '\nPrecision:' + str(precision.round(3)) + '\nRecall:' + str(recall.round(3)))
    pr_titles.append('Language pair:' + lp.upper())
    threshold = df_eval_results[df_eval_results['language_pair']==lp].iloc[0]['threshold']
    if LLM:
        file_prefix = lp + '_' + DATA_SPLIT + '_llm' 
    else:
        file_prefix = lp + '_' + DATA_SPLIT + '_' + str(seed) 
    if lp == 'en-ja' and enja_separate:
        folder = EXPERIMENT_GROUP_NAME + '_enja'
    else:
        folder = EXPERIMENT_GROUP_NAME
    for file in os.listdir(os.path.join(PREDICTIONS_DIR, 'ced_data', folder)):
        if file.startswith(file_prefix) and file.endswith('.csv'):
            # found predictions
            df_preds = pd.read_csv(os.path.join(PREDICTIONS_DIR, 'ced_data', folder, file))
            scores = df_preds['score'].to_numpy()
            scores = 1 - scores
            preds_probs.append(scores)
            binary_scores = scores > threshold
            binary_scores = binary_scores.astype('int')
            preds.append(binary_scores)
    targets.append(1 - load_ced_test_data(lp)['score'].to_numpy())

print(len(preds))
print(len(targets))

In [ ]:
fig = create_confusion_matrix_plot(CONFUSION_MATRIX_NAME, cm_titles, preds, targets)

In [ ]:
fig = create_precision_recall_curve(PR_PLOT_NAME, pr_titles, preds_probs, targets)